# Import definitions

In [1]:
!pip3 install openpyxl

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 kB 5.3 MB/s eta 0:00:00a 0:00:01
  Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config f

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
from preprocess_indexqual import preprocess, process_segment
from torch.nn import BCEWithLogitsLoss, BCELoss
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
from transformers import RobertaModel, RobertaForSequenceClassification, RobertaTokenizer, AdamW, RobertaForQuestionAnswering
from tqdm import tqdm, trange
import gc
import os
from balance_sampler import MultilabelBalancedRandomSampler
import shutil

# use CUDA when available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("using:", device)

# clean
torch.cuda.empty_cache()
gc.collect()

# define max length
max_length = 512


class MultilabelClassification(nn.Module):
    def __init__(self, bert_model, num_labels, hidden_dropout_prob=0.1, hidden_size=768):
        super(MultilabelClassification, self).__init__()

        self.config = {}
        self.bert_model = bert_model

        self.dense = nn.Linear(hidden_size, hidden_size)
        self.double_dense = nn.Linear(hidden_size * 2, hidden_size * 2)
        self.dropout = nn.Dropout(hidden_dropout_prob)
        self.classifier = nn.Linear(hidden_size * 2, num_labels)

    def forward(self, a_input_ids, a_input_mask, b_input_ids, b_input_mask):
        x1 = self.bert_model(a_input_ids, attention_mask=a_input_mask)
        x2 = self.bert_model(b_input_ids, attention_mask=b_input_mask)

        x1 = self.dropout(x1.last_hidden_state)
        x1 = x1[:, 0, :]  # take <s> token (equiv. to [CLS])

        x2 = self.dropout(x2.last_hidden_state)
        x2 = x2[:, 0, :]  # take <s> token (equiv. to [CLS])

        x = torch.cat((x1, x2), dim=1)

        x = self.double_dense(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.classifier(x)

        return x


def model_init():
    torch.cuda.empty_cache()
    gc.collect()
    bert = RobertaModel.from_pretrained("pdelobelle/robbert-v2-dutch-base", add_pooling_layer=False)

    for param in bert.parameters():
        param.requires_grad = False

    model = MultilabelClassification(bert, 15)
    return model


# TODO remove testing code
tokenizer = RobertaTokenizer.from_pretrained('pdelobelle/robbert-v2-dutch-base')  # tokenizer
context = tokenizer("Hallo, ik ben coen", truncation=True,
                                    max_length=max_length,
                                    padding=True, return_tensors='pt')
sentence = tokenizer("Ik vind het hier fijn, omdat ze goed verzorgen", truncation=True,
                                    max_length=max_length,
                                    padding=True, return_tensors='pt')
test_model = model_init()
# print(test_model)
test_model(context.input_ids, context.attention_mask, sentence.input_ids, sentence.attention_mask)

using: cpu


Some weights of the model checkpoint at pdelobelle/robbert-v2-dutch-base were not used when initializing RobertaModel: ['lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tensor([[ 0.1204,  0.2089, -0.0885, -0.1668,  0.2153,  0.1966, -0.2311,  0.0215,
          0.0377,  0.1871, -0.0138,  0.0352, -0.0158, -0.1426, -0.0133]],
       grad_fn=<AddmmBackward0>)

# Preprocessing

In [ ]:
# print start info
print("starting to train BERT with INDEXQUAL data")
print("preprocessing ...")

from_cache = True

# load preprocessed data
if from_cache:
    df = pd.read_csv('preprocessed.csv')
    print(len(df))
else:
    classes, df = preprocess(mode="")

    # to csv
    df.to_csv('preprocessed.csv', index = False, header=True)
    
# show data
print(df.head())

# select label columns
cols = df.columns
label_cols = list(cols[3:])
num_labels = len(label_cols)
print('Label columns: ', label_cols)
classes = label_cols

# set header for all label columns
df['labels'] = list(df[label_cols].values)
df.head()

# get input and outputs
labels = list(df.labels.values)
sentences = list(df.sentence.values)
print(len(sentences))
contexts = list(df.precursor.values)
print(len(contexts))

In [4]:
# tokenize data
tokenizer = RobertaTokenizer.from_pretrained('pdelobelle/robbert-v2-dutch-base')  # tokenizer
encodings = tokenizer.batch_encode_plus(sentences, truncation=True,
                                    max_length=max_length,
                                    padding=True)
print('tokenizer outputs: ', encodings.keys())

context_encodings = tokenizer.batch_encode_plus(contexts, truncation=True,
                                    max_length=max_length,
                                    padding=True)
print('tokenizer outputs: ', context_encodings.keys())

tokenizer outputs:  dict_keys(['input_ids', 'attention_mask'])
tokenizer outputs:  dict_keys(['input_ids', 'attention_mask'])


In [ ]:
# preparing data format for training
input_ids = encodings['input_ids']  # tokenized and encoded sentences
attention_masks = encodings['attention_mask']  # attention masks
context_input_ids = context_encodings['input_ids']  # tokenized and encoded sentences
context_attention_masks = context_encodings['attention_mask']  # attention masks
print(len(input_ids))
print(len(attention_masks))
print(len(context_input_ids))
print(len(context_attention_masks))
print(len(labels))

# Identifying all samples that include only one label, this avoids issues with stratification
label_counts = df.labels.astype(str).value_counts()
one_freq = label_counts[label_counts == 1].keys()
one_freq_idxs = sorted(list(df[df.labels.astype(str).isin(one_freq)].index), reverse=True)
print('df label indices with only one instance: ', one_freq_idxs)

# Gathering single instance inputs to force into the training set after stratified split
one_freq_input_ids = [input_ids.pop(i) for i in one_freq_idxs]
one_freq_attention_masks = [attention_masks.pop(i) for i in one_freq_idxs]
one_freq_context_ids = [context_input_ids.pop(i) for i in one_freq_idxs]
one_freq_context_attention_masks = [context_attention_masks.pop(i) for i in one_freq_idxs]
one_freq_labels = [labels.pop(i) for i in one_freq_idxs]

# Use train_test_split to split our data into train and validation sets
train_inputs, validation_inputs, train_labels, validation_labels, train_masks, validation_masks, train_context_inputs, validation_context_inputs, train_context_masks, validation_context_masks = train_test_split(
        input_ids, labels, attention_masks, context_input_ids, context_attention_masks,
        random_state=2020, test_size=0.15, stratify=labels)

# Add one frequency data to train data
train_inputs.extend(one_freq_input_ids)
train_masks.extend(one_freq_attention_masks)
train_context_inputs.extend(one_freq_context_ids)
train_context_masks.extend(one_freq_context_attention_masks)
train_labels.extend(one_freq_labels)

# Convert all of our data into torch tensors, the required datatype for our model
train_inputs = torch.tensor(train_inputs)
train_masks = torch.tensor(train_masks)
train_context_inputs = torch.tensor(train_context_inputs)
train_context_masks = torch.tensor(train_context_masks)
train_labels = torch.tensor(train_labels)

validation_inputs = torch.tensor(validation_inputs)
validation_masks = torch.tensor(validation_masks)
validation_context_inputs = torch.tensor(validation_context_inputs)
validation_context_masks = torch.tensor(validation_context_masks)
validation_labels = torch.tensor(validation_labels)

In [12]:
validation_labels.size()

torch.Size([936, 15])

In [7]:
# Select a batch size for training. For fine-tuning with XLNet, the authors recommend a batch size of 32, 48,
# or 128. We will use 32 here to avoid memory issues.
batch_size = 8

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because,
# unlike a for loop, with an iterator the entire dataset does not need to be loaded into memory
train_data = TensorDataset(train_inputs, train_masks, train_context_inputs, train_context_masks, train_labels)
train_sampler = MultilabelBalancedRandomSampler(train_labels)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_context_inputs, validation_context_masks, validation_labels)
validation_sampler = SequentialSampler(validation_labels)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

# Training

In [ ]:
# Init model
model = model_init()
model.to(device)
    
# TODO add seed constant

param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5, correct_bias=True)

In [15]:
def save_model():
    best_name = 'bert_model_multi_label_indexqual_' + str(val_f1_accuracy)
    dic = zip(range(0, len(classes)), classes)
    torch.save(model.state_dict(), best_name)
    
    return best_name

In [16]:
# Store our loss and accuracy for plotting
train_loss_set = []
train_loss_per_epoch = []
valid_loss_per_epoch = []
valid_acc_set = []
best_valid_f1 = 0
best_name = ""

In [ ]:
# Number of training epochs (authors recommend between 2 and 4)
epochs = 2

# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):
    # Set our model to training mode (as opposed to evaluation mode)
    model.train()

    # Tracking variables
    tr_loss = 0  # running loss
    nb_tr_examples, nb_tr_steps = 0, 0

    # Train the data for one epoch
    with tqdm(total=len(train_dataloader), position=0, leave=True) as pbar:
        for i, batch in enumerate(tqdm(train_dataloader, position=0, leave=True)):
            # Add batch to GPU
            batch = tuple(t.to(device) for t in batch)

            # Unpack the inputs from our dataloader
            b_input_ids, b_input_mask, c_input_ids, c_input_mask, b_labels = batch

            # Clear out the gradients (by default they accumulate)
            optimizer.zero_grad()

            # Forward pass for multilabel classification
            logits = model(c_input_ids, c_input_mask, b_input_ids, b_input_mask)
            loss_func = BCEWithLogitsLoss()

            loss = loss_func(logits.view(-1, num_labels),
                             b_labels.type_as(logits).view(-1, num_labels))  # convert labels to float for calculation
            train_loss_set.append(loss.item())

            # Backward pass
            loss.backward()
            # Update parameters and take a step using the computed gradient
            optimizer.step()
            # scheduler.step()
            # Update tracking variables
            tr_loss += loss.item()
            nb_tr_examples += b_input_ids.size(0)
            nb_tr_steps += 1
            
            # update progress bar
            pbar.update()

    print("Train loss: {}".format(tr_loss / nb_tr_steps))
    train_loss_per_epoch.append(tr_loss / nb_tr_steps)
    
    ###############################################################################

    # Validation

    # Put model in evaluation mode to evaluate loss on the validation set
    model.eval()

    # Variables to gather full output
    logit_preds, true_labels, pred_labels, tokenized_texts = [], [], [], []

    # Tracking variables
    vd_loss = 0  # running loss
    nb_vd_steps = 0
    
    # Predict
    with tqdm(total=len(validation_dataloader), position=0, leave=True) as pbar:
        for i, batch in enumerate(tqdm(validation_dataloader, position=0, leave=True)):
            batch = tuple(t.to(device) for t in batch)
            # Unpack the inputs from our dataloader
            b_input_ids, b_input_mask, c_input_ids, c_input_mask, b_labels = batch

            with torch.no_grad():
                # Forward pass
                b_logit_pred = model( c_input_ids, c_input_mask, b_input_ids, b_input_mask)

                loss = loss_func(b_logit_pred.view(-1, num_labels),
                     b_labels.type_as(b_logit_pred).view(-1, num_labels))  # convert labels to float for calculation
                vd_loss += loss.item()
                nb_vd_steps += 1

                pred_label = torch.sigmoid(b_logit_pred)
                b_logit_pred = b_logit_pred.detach().cpu().numpy()
                pred_label = pred_label.to('cpu').numpy()
                b_labels = b_labels.to('cpu').numpy()

            tokenized_texts.append(b_input_ids)
            logit_preds.append(b_logit_pred)
            true_labels.append(b_labels)
            pred_labels.append(pred_label)
            
            # update progress bar
            pbar.update()
    
    print("Validation loss: {}".format(vd_loss / nb_vd_steps))
    valid_loss_per_epoch.append(vd_loss / nb_vd_steps)
        
    # Flatten outputs
    pred_labels = [item for sublist in pred_labels for item in sublist]
    true_labels = [item for sublist in true_labels for item in sublist]
    
    # Calculate Accuracy
    threshold = 0.50
    pred_bools = [pl > threshold for pl in pred_labels]
    true_bools = [tl == 1 for tl in true_labels]
    val_f1_accuracy = f1_score(true_bools, pred_bools, average='micro') * 100
    val_flat_accuracy = accuracy_score(true_bools, pred_bools) * 100

    valid_acc_set.append(val_f1_accuracy)
    print('F1 Validation Accuracy: ', val_f1_accuracy)
    print('Flat Validation Accuracy: ', val_flat_accuracy)   
    
    if val_f1_accuracy > best_valid_f1:
        if os.path.exists(best_name) and os.path.isdir(best_name):
            shutil.rmtree(best_name)
        best_valid_f1 = val_f1_accuracy
        best_name = save_model()

# Simple predictions

In [11]:
# from transformers import pipeline
# nlp = pipeline('text-classification', tokenizer="pdelobelle/robbert-v2-dutch-base", model=best_name, return_all_scores=True)
# model = RobertaForSequenceClassification.from_pretrained(best_name, num_labels=len(classes))
# model = model.to('cuda')

In [12]:
# print(nlp("Ik verwacht dat ik geholpen wordt")) # expectations
# print(nlp("Het is toch kleinschalig wonen.")) # context
# print(nlp("hij komt van herstelzorg.")) # past experiences
# print(nlp("Ik denk dat de activiteiten misschien dat hier aangeboden zijn niet zo zijn als dat ik gedacht had..")) # expectations

# Confusion matrix

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, confusion_matrix

# Variables to gather full output
logit_preds, true_labels, pred_labels, tokenized_texts = [], [], [], []

# Use original distribution for evaluation (instead of a balanced distribution)
validation_sampler = SequentialSampler(validation_labels)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

# Predict
for i, batch in enumerate(validation_dataloader):
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, c_input_ids, c_input_mask, b_labels = batch
    with torch.no_grad():
        # Forward pass
        b_logit_pred = model( c_input_ids, c_input_mask, b_input_ids, b_input_mask)
        pred_label = torch.sigmoid(b_logit_pred)

        b_logit_pred = b_logit_pred.detach().cpu().numpy()
        pred_label = pred_label.to('cpu').numpy()
        b_labels = b_labels.to('cpu').numpy()

    tokenized_texts.append(b_input_ids)
    logit_preds.append(b_logit_pred)
    true_labels.append(b_labels)
    pred_labels.append(pred_label)

# Flatten outputs
true_labels = [item for sublist in true_labels for item in sublist]
pred_labels = [item for sublist in pred_labels for item in sublist]

# Calculate Accuracy
threshold = 0.50
pred_bools = [pl > threshold for pl in pred_labels]
true_bools = [tl == 1 for tl in true_labels]
val_f1_accuracy = f1_score(true_bools, pred_bools, average='micro') * 100
val_flat_accuracy = accuracy_score(true_bools, pred_bools) * 100

print('F1 Validation Accuracy: ', val_f1_accuracy)
print('Flat Validation Accuracy: ', val_flat_accuracy)   

# calculate predicted class for single-label CFM
true_labels_single = np.argmax(true_labels, axis=1)
pred_labels_single = np.argmax(pred_labels, axis=1)

cm = confusion_matrix(true_labels_single, pred_labels_single)
cm_df = pd.DataFrame(cm, index=classes, columns=classes)
cm_df.index.name = 'Actual'
cm_df.columns.name = 'Predicted'

print(cm_df)

# calculate predicted class for multi-label CFM
true_labels_multi = np.round(true_labels)
pred_labels_multi = np.round(pred_labels)

cm_multi = multilabel_confusion_matrix(true_labels_multi, pred_labels_multi)

for i in range(0, len(classes)):
    cm_multi[i] = np.fliplr(np.rot90(cm_multi[i]))

print(cm_multi)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

figsize = (20, 8)

fig = plt.figure(figsize=figsize)

plt.subplot(1, 2, 1)
plt.plot(train_loss_per_epoch, 'g')
plt.plot(valid_loss_per_epoch, 'b')
plt.grid(True)

plt.subplot(1, 2, 2)
sns.heatmap(cm_df, annot=cm, fmt='', cmap="Blues", annot_kws={"size": 10})
plt.show()

In [ ]:
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
import seaborn as sns

fig, axs = plt.subplots(2,2)
fig.set_size_inches(12, 12)

def calc_accuracy(i):
    true_1 = cm_multi[i,0,0] + cm_multi[i,1,1]
    false_1 = cm_multi[i,1,0] + cm_multi[i,0,1]
    return true_1 / (true_1 + false_1)

def calc_sensitivity(i):
    tp = cm_multi[i,1,1]
    fn = cm_multi[i,1,0]
    return round(tp / (tp + fn), 2)

def calc_specificity(i):
    tn = cm_multi[i,0,0]
    fp = cm_multi[i,0,1]
    return round(tn / (tn + fp), 2)

tick_labels = ['Present', 'Absent']
xlabel = "Manually coded"
ylabel = "As predicted by text mining"

plt.subplot(2, 2, 1)
sns.heatmap(cm_multi[0], annot=cm_multi[0], fmt='', cmap="Blues", annot_kws={"size": 15}, cbar=False, xticklabels=tick_labels, yticklabels=tick_labels)
# axs[0, 0].set_title(classes[0] + " (" + str(calc_sensitivity(0)) + ", " + str(calc_specificity(0)) + ")")
axs[0, 0].set_title('Experienced QoC', fontsize=20, fontweight='bold')
plt.xlabel(xlabel, fontsize=12)
plt.ylabel(ylabel, fontsize=12)

plt.subplot(2, 2, 2)
sns.heatmap(cm_multi[1], annot=cm_multi[1], fmt='', cmap="Blues", annot_kws={"size": 15}, cbar=False, xticklabels=tick_labels, yticklabels=tick_labels)
# axs[0, 1].set_title(classes[1] + " (" + str(calc_sensitivity(1)) + ", " + str(calc_specificity(1)) + ")")
axs[0, 1].set_title('Experiences', fontsize=20, fontweight='bold')
plt.xlabel(xlabel, fontsize=12)
plt.ylabel(ylabel, fontsize=12)

plt.subplot(2, 2, 3)
sns.heatmap(cm_multi[2], annot=cm_multi[2], fmt='', cmap="Blues", annot_kws={"size": 15}, cbar=False, xticklabels=tick_labels, yticklabels=tick_labels)
# axs[1, 0].set_title(classes[2] + " (" + str(calc_sensitivity(2)) + ", " + str(calc_specificity(2)) + ")")
axs[1, 0].set_title('Expectations', fontsize=20, fontweight='bold')
plt.xlabel(xlabel, fontsize=12)
plt.ylabel(ylabel, fontsize=12)

plt.subplot(2, 2, 4)
sns.heatmap(cm_multi[3], annot=cm_multi[3], fmt='', cmap="Blues", annot_kws={"size": 15}, cbar=False, xticklabels=tick_labels, yticklabels=tick_labels)
# axs[1, 1].set_title(classes[3] + " (" + str(calc_sensitivity(3)) + ", " + str(calc_specificity(3)) + ")")
axs[1, 1].set_title('Context', fontsize=20, fontweight='bold')
plt.xlabel(xlabel, fontsize=12)
plt.ylabel(ylabel, fontsize=12)

In [25]:
import json

filepath = 'data.txt'

context_text = ''

obj_list = []

# with open(filepath, encoding='cp1252') as f:
with open(filepath) as f:
    lines = f.readlines()

    for i in range(0, len(lines)):
        context = tokenizer(context_text, truncation=True,
                                        max_length=max_length,
                                        padding=True, return_tensors='pt')
        sentence = tokenizer(lines[i], truncation=True,
                                        max_length=max_length,
                                        padding=True, return_tensors='pt')

        context.to(device)
        sentence.to(device)
        
        result = model(context.input_ids, context.attention_mask, sentence.input_ids, sentence.attention_mask)
        
        pred_bools = [pl > threshold for pl in result.detach().cpu().numpy()][0]
        
        obj = {
            "text": lines[i],
            "labels": pred_bools.tolist()
        }
        
        obj_list.append(obj)

        context_text += ' ' + lines[i]
        
        context_split = context_text.split(' ')
        if len(context_split) > 511:
            context_split = context_split[-511:]
            context_text = ' '.join(context_split)
            
        
json_dump = json.dumps(obj_list)

print(json_dump)

with open("data.json", "w") as f:
    f.write(json_dump)
    
print("Done!")

FileNotFoundError: [Errno 2] No such file or directory: 'data.txt'